# SBERT를 이용한 챗봇 만들기

* SBERT(Sentence BERT): 기본적으로 BERT의 문장 임베딩의 성능을 우수하게 개선시킨 모델
* sentence_transformers : SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지
* 런타임 유형을 GPU로 설정해주세요!!

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 5.3 MB 58.0 MB/s 
     |████████████████████████████████| 1.3 MB 46.3 MB/s 
     |████████████████████████████████| 163 kB 70.3 MB/s 
     |████████████████████████████████| 7.6 MB 55.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4f9e16e51fa460121f62ea480542ee4a7bc2f98951f9ea0221de937dd6be6f2b
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
import pandas as pd
import urllib.request

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
# 데이터 로드하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
# 데이터 건수
len(train_data)

11823

In [6]:
import numpy as np
np.unique(train_data.label)

array([0, 1, 2])

In [7]:
# 일상다반사: 0, 이별(부정): 1, 사랑(긍정): 2로 라벨링
train_data[train_data.label == 0][:5]

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [8]:
# 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델
# Sentence Tranformer로 로드할 수 있는 모델 리스트: https://huggingface.co/models?library=sentence-transformers
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [9]:
# train_data['Q']에 대해 모든 문장 임베딩 값을 구함
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)
train_data.head()

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.20179537, -0.034437943, 1.5395736, 0.010697..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.07716601, -0.034278184, 0.86244196, 0.02636..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.10445247, -0.012432299, 1.0132881, 0.022501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760744, -0.046716824, 0.89369464, 0.02104..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002862, 0.03196111, 1.491543, 4.3268716e..."


In [10]:
from numpy import dot
from numpy.linalg import norm

In [11]:
# 두 개의 벡터로부터 코사인 유사도를 구하는 함수
def cos_sim(A, B):
  return dot(A, B) / (norm(A) * norm(B))

In [12]:
# 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열
# 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 
# 코사인 유사도 값이 가장 높은 질문 샘플을 찾아내고, 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴함
def get_similar_answer(input):
  embedding = model.encode(input)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
  return train_data.loc[train_data['score'].idxmax()]['A']

In [13]:
get_similar_answer('너무 배고프네')

'뭐 좀 챙겨드세요.'

In [14]:
get_similar_answer('치킨 먹고 싶다')

'맛있죠!'

In [15]:
get_similar_answer('여행가고 싶네')

'좋은 취미네요.'

In [16]:
get_similar_answer('커피 한잔 먹을까?')

'좋은 시간 보내시길 바라요.'

In [17]:
get_similar_answer('너는 누구니?')

'저는 위로봇입니다.'

In [18]:
get_similar_answer('너는 몇살이니?')

'나이 상관없어요.'

In [19]:
get_similar_answer('공부는 왜 해야할까?')

'공부하면 더 많은 선택을 할 수 있죠.'